

This is a go at trying to explore the open-data of energy, this will be a on going gist.
0.01 Germany

First we explore macro data points. What is more macro then goverment policy, rules and subsidies.

    Germany: Nuclear power plants to close by 2022 https://www.bbc.com/news/world-europe-13592208.

Q1. how much nuclear is germeny genrating and consuming?
Q2. how many plants and are is being decommissioned?

    Legislative support for the Energiewende was passed in late 2010 and includes greenhouse gas. a renewable energy target of 60% by 2050. Energiewende aimes to curb CO2 emissions by 80–95%. https://www.bmwi.de/Redaktion/EN/Publikationen/vierter-monitoring-bericht-energie-der-zukunft-kurzfassung.pdf

Q1. how much renewable is genrated and consumed?
Q2. which plants will be de-commisioned?


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from IPython.display import display as di
import geopy
import folium
from shapely.geometry import Point, Polygon
from geopy.geocoders import Nominatim
import geopandas as gpd
import json

from branca import colormap as cm

from minio import Minio
from minio.error import (ResponseError, BucketAlreadyOwnedByYou,
                         BucketAlreadyExists)
                         



for styleing, things to add:

    snippits
    hiding
    table of contents

todo:

    scrap cities population
    draw polgram around the cities
        done
    draw Choropleth for other, Nuclear, Fossil-fuels and Renewable-energy
        done
    write about the missing values and try to invastgate it and estmate it
    go through all the columns descrbing them, and seeing potential in drawing
    draw companies


41 million households 82 million people 130 TWh of electricity annually Installed capacity 198.45 GW

reviewing the columns first we review the data. columns:

    energy_source_level_1
    energy_source_level_2




In [ ]:
minioClient = Minio('play.min.io',
                access_key='Q3AM3UQ867SPQQA43P2F',
                secret_key='zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG')
prefix = 'germanyenergy/'
try:
    data = minioClient.get_object('workflow', prefix +'output.geojson')
    with open('output.geojson', 'wb') as file_data:
        for d in data.stream(32*1024):
            file_data.write(d)
except ResponseError as err:
    print(err)
    
geo_data=r"./output.geojson"

def cholo( ele, geo_data):
    
    de = gpd.read_file(geo_data,)
    
    ele = de.set_index('ID_3')[ele]
    colormap = cm.LinearColormap(
    ['white' ,'yellow', 'orange', 'red'],
    index=[0, 100, 1000, 8000]
    ,vmin=0,vmax=8000)
    
    m = folium.Map(
        location=[51.1657, 10.4515],
        zoom_start=6,
        min_zoom=6, 
        max_zoom=6
    )
    folium.GeoJson(
        geo_data,
        name='germany init drawing',
        style_function=lambda feature: {
            'fillColor': colormap(ele[feature['properties']['ID_3']]),
            'color': 'black',
            'weight': 1,
            'dashArray': '5, 5',
            'fillOpacity': 0.9,
    }
).add_to(m)
    
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}
    NIL = folium.features.GeoJson(
        "./output.geojson",
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NAME_3','total','ID_3'],
            aliases=['Neighborhood: ','Resident foreign population in %: ','ID:  '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    m.add_child(NIL)
    m.keep_in_front(NIL)
    folium.LayerControl().add_to(m)
    display(m)


cholo('total', geo_data)

In [ ]:


'Renewable energy'
'Nuclear'
'Fossil fuels'
cholo('Nuclear',  geo_data)



In [ ]:
cholo( 'Fossil fuels',  geo_data)

In [ ]:
cholo( 'Renewable energy',  geo_data)
